In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# Метод $k$ ближайших соседей (kNN)

В этой части работы вы:
- научитесь готовить данные к построению модели (предобработка, или препроцессинг данных);
- познакомитесь с методами ближайших соседей для задач классификации, реализованными в библиотеке scikit-learn ;
- научитесь оценивать качество модели с помощью отложенной выборки.

## 1. Подключитесь к одному из предложенных наборов данных:
- Adult income;
- Telecom churn;
- Cardiovascular disease
- Red wine quality

или выберите свой. Разберитесь в том, как устроен ваш датасет и какова постановка задачи.

In [ ]:
# My code
df = pd.read_csv('../input/adult-income-dataset/adult.csv')
df.head()


## 2. Проведите предварительный разведочный анализ выборки (типы данных, статистики, распределения, визуализация, корреляционный анализ).

In [ ]:
# Here we go!
df.keys()

In [ ]:
df.dtypes

## 3. Проведите необходимую предобработку данных (preprocessing). 

Для построения моделей с помощью метрических методов **все признаки должны быть закодированы числами**. Полезными будут следующие методы библиотеки Pandas:
- map() –– для перекодировки категориальной переменной числовыми метками;
- get_dummies() –– для создания нескольких бинарных признаков на основе категориального.

Также может потребоваться масштабирование данных (scaling). Воспользуйтесь классом StandardScaler библиотеки Scikit-learn:
```
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)
```
Заметим, что целевую переменную масштабировать, как правило, не требуется.

In [ ]:
# Hmm... It is not that easy!
df['income'] = df['income'].map({'<=50K': 0, '>50K': 1})

In [ ]:
X = df[['age', 'educational-num', 'hours-per-week']]
y = df['income']
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)


## 4. Разбейте набор данных на обучающую и валидационную (тестовую) выборки с помощью метода train_test_split в соотношении 75 на 25.

In [ ]:
# You need just two lines of code :)
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X,
                                                     y,
                                                     test_size = 0.25,
                                                     random_state = 8)

## 5. Обучите алгоритм классификации kNeighborsClassifier или регрессии KNeighborsRegressor. 

Оцените качество каждой модели на валидационной выборке с помощью
- **accuracy_score** для классификации;
- **mean_squared_error** для регрессии.
Сравните результаты и сделайте выводы.

In [ ]:
# Keep up good work!
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)

In [ ]:
y_pred = knn.predict(X_valid)

In [ ]:
from sklearn.metrics import accuracy_score, mean_squared_error
print(accuracy_score(y_valid, y_pred))

# Настройка гиперпараметров

В данной части работы вы научитесь настраивать параметр **n_neighbors** алгоритма kNN с помощью перекрёстной проверки (кросс-валидации).

## 6. Создайте генератор разбиений, который перемешивает выборку перед созданием блоков (shuffle=True).

Число блоков **n_splits** равно 5. Задайте также параметр **random_state** для воспроизводимости результатов. Например:
```
kf = KFold(n_splits=5, shuffle=True, random_state=42)
```
Найдите показатель качества модели kNN на кросс-валидации. Подумайте, приемлемо ли использование вашей меры (метрики) качества в данной задаче? При необходимости пересчитайте качество с помощью другой метрики из [списка](https://scikit-learn.org/stable/modules/model_evaluation.html).

In [ ]:
# My code
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
kf = KFold(n_splits=5, shuffle=True, random_state=8)
knn = KNeighborsClassifier(n_neighbors=5)
scores = cross_val_score(knn, X,y, 
                         cv=kf, scoring= 'accuracy')
print(scores)
mean_score = scores.mean()
print(mean_score)

## 7. Осуществите кросс-валидацию модели при числе соседей от 1 до 50.

Используйте **GridSearchCV**. При каком $k$ качество получилось наилучшим? Чему равна
эта оценка качества? Постройте график значений метрики в зависимости от $k$
(matplotlib.pyplot.plot()).

In [ ]:
# My code
from sklearn.model_selection import GridSearchCV
knn_params = {'n_neighbors': np.arange(1, 51)}
knn_grid = GridSearchCV(knn, 
                        knn_params, 
                        scoring='accuracy',
                        cv=5) # или cv=kf
knn_grid.fit(X_train, y_train)

In [ ]:
knn_grid.best_estimator_

In [ ]:
knn_grid.best_score_

In [ ]:
knn_grid.best_params_

In [ ]:
knn_grid.cv_results_['std_test_score']

In [ ]:
plt.pyplot.plot(np.arange(1, 51),knn_grid.cv_results_['mean_test_score'])

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 46, weights = 'distance')

knn_params ={'p': np.linspace(1, 10, 20, endpoint = True)}
knn_grid = GridSearchCV(knn, 
                        knn_params, 
                        scoring='accuracy',
                        cv=5)

knn_grid.fit(X_train, y_train)

In [ ]:
knn_grid.best_params_

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 46, weights = 'distance', p = knn_grid.best_params_['p'] )
scores = cross_val_score(knn, X,y, 
                         cv=5, scoring= 'accuracy')

print(scores.mean())

In [ ]:
#NearestCentroid
#from sklearn.neighbors import NearestCentroid

#clf = NearestCentroid()
#clf.fit(X_train,y_train)

#y_pred = clf.predict(np.array(y_valid).reshape(-1,1))
#print(accuracy_score(y_valid, y_pred))